In [23]:
import pandas as pd
import numpy as np
import math

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import StratifiedKFold


from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri


import matplotlib.pyplot as plt

import SMOTE
import feature_selector

import warnings
warnings.filterwarnings("ignore")

In [15]:
df = pd.read_csv('/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/ant_updated.csv')

In [16]:
fs = feature_selector.featureSelector()

In [17]:
df_new,_selected,selected = fs.cfs_bfs(df)
print(selected)

fix
['lt', 'rexp', 'ns', 'nd', 'exp', 'entropy', 'ndev', 'nf', 'la', 'ld', 'age'] [0.07, 0.1, 0.1, 0.1, 0.11, 0.11, 0.12, 0.12, 0.12, 0.12, 0.12]
['lt', 'rexp', 'ns', 'nd', 'exp', 'entropy', 'ndev', 'fix']


In [ ]:
subset = []
for i in range(10):
    df_new,_selected,selected = fs.cfs_bfs(df)
    subset.append(_selected)
    print(selected)
subset_df = pd.DataFrame(subset, columns = ['$wmc', '$dit', '$noc', '$cbo', '$rfc', '$lcom', '$ca', '$ce', '$npm',
       '$lcom3', '$loc', '$dam', '$moa', '$mfa', '$cam', '$ic', '$cbm', '$amc',
       '$max_cc', '$avg_cc'])
subset_df.sum()

In [19]:
y = df.bug.values
X = df.drop(labels = ['bug'],axis = 1)

AttributeError: 'DataFrame' object has no attribute 'bug'

In [18]:
subset = []
kf = StratifiedKFold(n_splits = 10)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    train_df = X_train
    train_df['bug'] = y_train
    test_df = X_test
    test_df['bug'] = y_test
    print("++++++++++++++++++++++++++++++++++++++++++++")
    # using CFS implemented
    train_df_new,_selected,selected = fs.cfs_bfs(df)
    subset.append(_selected)
    train_y = train_df_new.bug.values
    train_X = train_df_new.drop(labels = ['bug'],axis = 1)
    test_df_new = test_df[selected]
    test_y = test_df_new.bug.values
    test_X = test_df_new.drop(labels = ['bug'],axis = 1)
    clf = LogisticRegression(penalty='l1',class_weight = 'balanced')
    clf.fit(train_X,train_y)
    predicted = clf.predict(test_X)
    print("Using Python CFS selected features")
    print(round(metrics.f1_score(test_y, predicted,average='weighted'),2))
    
    #using Weka CFS
    selected = ['wmc','cbo','rfc','lcom','loc','dam','cam','bug']
    #selected = ['ce','cbo','bug']
    train_df_new = train_df[selected]
    train_y = train_df_new.bug.values
    train_X = train_df_new.drop(labels = ['bug'],axis = 1)
    test_df_new = test_df[selected]
    test_y = test_df_new.bug.values
    test_X = test_df_new.drop(labels = ['bug'],axis = 1)
    clf = LogisticRegression(penalty='l1',class_weight = 'balanced')
    clf.fit(train_X,train_y)
    predicted = clf.predict(test_X)
    print("Using Weka CFS selected features")
    print(round(metrics.f1_score(test_y, predicted,average='weighted'),2))
subset_df = pd.DataFrame(subset, columns = ['$wmc', '$dit', '$noc', '$cbo', '$rfc', '$lcom', '$ca', '$ce', '$npm',
       '$lcom3', '$loc', '$dam', '$moa', '$mfa', '$cam', '$ic', '$cbm', '$amc',
       '$max_cc', '$avg_cc'])

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [ ]:
subset_df.sum()

In [ ]:
subset = []
kf = StratifiedKFold(n_splits = 10)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    train_df = X_train
    train_df['bug'] = y_train
    test_df = X_test
    test_df['bug'] = y_test
    df_new = train_df[['wmc','cbo','rfc','lcom','loc','dam','cam','bug']]
    train_y = df_new.bug.values
    train_X = df_new.drop(labels = ['bug'],axis = 1)
    test_df = test_df[['wmc','cbo','rfc','lcom','loc','dam','cam','bug']]
    test_y = test_df.bug.values
    test_X = test_df.drop(labels = ['bug'],axis = 1)
    clf = LogisticRegression(penalty='l1',class_weight = 'balanced')
    clf.fit(train_X,train_y)
    predicted = clf.predict(test_X)
    print(classification_report(test_y, predicted))
subset_df = pd.DataFrame(subset, columns = ['$wmc', '$dit', '$noc', '$cbo', '$rfc', '$lcom', '$ca', '$ce', '$npm',
       '$lcom3', '$loc', '$dam', '$moa', '$mfa', '$cam', '$ic', '$cbm', '$amc',
       '$max_cc', '$avg_cc'])

In [5]:
import numpy as np


def merit_calculation(X, y):
    """
    This function calculates the merit of X given class labels y, where
    merits = (k * rcf)/sqrt(k+k*(k-1)*rff)
    rcf = (1/k)*sum(su(fi,y)) for all fi in X
    rff = (1/(k*(k-1)))*sum(su(fi,fj)) for all fi and fj in X

    Input
    ----------
    X: {numpy array}, shape (n_samples, n_features)
        input data
    y: {numpy array}, shape (n_samples,)
        input class labels

    Output
    ----------
    merits: {float}
        merit of a feature subset X
    """

    n_samples, n_features = X.shape
    rff = 0
    rcf = 0
    for i in range(n_features):
        fi = X[:, i]
        rcf += su_calculation(fi, y)
        for j in range(n_features):
            if j > i:
                fj = X[:, j]
                rff += su_calculation(fi, fj)
    rff *= 2
    merits = rcf / np.sqrt(n_features + rff)
    return merits


def cfs(X, y):
    """
    This function uses a correlation based heuristic to evaluate the worth of features which is called CFS

    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    y: {numpy array}, shape (n_samples,)
        input class labels

    Output
    ------
    F: {numpy array}
        index of selected features

    Reference
    ---------
    Zhao, Zheng et al. "Advancing Feature Selection Research - ASU Feature Selection Repository" 2010.
    """

    n_samples, n_features = X.shape
    F = []
    # M stores the merit values
    M = []
    while True:
        merit = -100000000000
        idx = -1
        for i in range(n_features):
            if i not in F:
                F.append(i)
                # calculate the merit of current selected features
                t = merit_calculation(X[:, F], y)
                print(F,t)
                if t > merit:
                    merit = t
                    idx = i
                F.pop()
        F.append(idx)
        M.append(merit)
        if len(M) > 5:
            if M[len(M)-1] <= M[len(M)-2]:
                if M[len(M)-2] <= M[len(M)-3]:
                    if M[len(M)-3] <= M[len(M)-4]:
                        if M[len(M)-4] <= M[len(M)-5]:
                            break
    return np.array(F)


In [6]:
# Written by Greg Ver Steeg (http://www.isi.edu/~gregv/npeet.html)

import scipy.spatial as ss
from scipy.special import digamma
from math import log
import numpy.random as nr
import numpy as np
import random


# continuous estimators

def entropy(x, k=3, base=2):
    """
    The classic K-L k-nearest neighbor continuous entropy estimator x should be a list of vectors,
    e.g. x = [[1.3],[3.7],[5.1],[2.4]] if x is a one-dimensional scalar and we have four samples
    """

    assert k <= len(x)-1, "Set k smaller than num. samples - 1"
    d = len(x[0])
    N = len(x)
    intens = 1e-10  # small noise to break degeneracy, see doc.
    x = [list(p + intens * nr.rand(len(x[0]))) for p in x]
    tree = ss.cKDTree(x)
    nn = [tree.query(point, k+1, p=float('inf'))[0][k] for point in x]
    const = digamma(N)-digamma(k) + d*log(2)
    return (const + d*np.mean(map(log, nn)))/log(base)


def mi(x, y, k=3, base=2):
    """
    Mutual information of x and y; x, y should be a list of vectors, e.g. x = [[1.3],[3.7],[5.1],[2.4]]
    if x is a one-dimensional scalar and we have four samples
    """

    assert len(x) == len(y), "Lists should have same length"
    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    intens = 1e-10  # small noise to break degeneracy, see doc.
    x = [list(p + intens * nr.rand(len(x[0]))) for p in x]
    y = [list(p + intens * nr.rand(len(y[0]))) for p in y]
    points = zip2(x, y)
    # Find nearest neighbors in joint space, p=inf means max-norm
    tree = ss.cKDTree(points)
    dvec = [tree.query(point, k+1, p=float('inf'))[0][k] for point in points]
    a, b, c, d = avgdigamma(x, dvec), avgdigamma(y, dvec), digamma(k), digamma(len(x))
    return (-a-b+c+d)/log(base)


def cmi(x, y, z, k=3, base=2):
    """
    Mutual information of x and y, conditioned on z; x, y, z should be a list of vectors, e.g. x = [[1.3],[3.7],[5.1],[2.4]]
    if x is a one-dimensional scalar and we have four samples
    """

    assert len(x) == len(y), "Lists should have same length"
    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    intens = 1e-10  # small noise to break degeneracy, see doc.
    x = [list(p + intens * nr.rand(len(x[0]))) for p in x]
    y = [list(p + intens * nr.rand(len(y[0]))) for p in y]
    z = [list(p + intens * nr.rand(len(z[0]))) for p in z]
    points = zip2(x, y, z)
    # Find nearest neighbors in joint space, p=inf means max-norm
    tree = ss.cKDTree(points)
    dvec = [tree.query(point, k+1, p=float('inf'))[0][k] for point in points]
    a, b, c, d = avgdigamma(zip2(x, z), dvec), avgdigamma(zip2(y, z), dvec), avgdigamma(z, dvec), digamma(k)
    return (-a-b+c+d)/log(base)


def kldiv(x, xp, k=3, base=2):
    """
    KL Divergence between p and q for x~p(x), xp~q(x); x, xp should be a list of vectors, e.g. x = [[1.3],[3.7],[5.1],[2.4]]
    if x is a one-dimensional scalar and we have four samples
    """

    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    assert k <= len(xp) - 1, "Set k smaller than num. samples - 1"
    assert len(x[0]) == len(xp[0]), "Two distributions must have same dim."
    d = len(x[0])
    n = len(x)
    m = len(xp)
    const = log(m) - log(n-1)
    tree = ss.cKDTree(x)
    treep = ss.cKDTree(xp)
    nn = [tree.query(point, k+1, p=float('inf'))[0][k] for point in x]
    nnp = [treep.query(point, k, p=float('inf'))[0][k-1] for point in x]
    return (const + d*np.mean(map(log, nnp))-d*np.mean(map(log, nn)))/log(base)


# Discrete estimators
def entropyd(sx, base=2):
    """
    Discrete entropy estimator given a list of samples which can be any hashable object
    """

    return entropyfromprobs(hist(sx), base=base)


def midd(x, y):
    """
    Discrete mutual information estimator given a list of samples which can be any hashable object
    """

    return -entropyd(list(zip(x, y)))+entropyd(x)+entropyd(y)


def cmidd(x, y, z):
    """
    Discrete mutual information estimator given a list of samples which can be any hashable object
    """

    return entropyd(list(zip(y, z)))+entropyd(list(zip(x, z)))-entropyd(list(zip(x, y, z)))-entropyd(z)


def hist(sx):
    # Histogram from list of samples
    d = dict()
    for s in sx:
        d[s] = d.get(s, 0) + 1
    return map(lambda z: float(z)/len(sx), d.values())


def entropyfromprobs(probs, base=2):
    # Turn a normalized list of probabilities of discrete outcomes into entropy (base 2)
    return -sum(map(elog, probs))/log(base)


def elog(x):
    # for entropy, 0 log 0 = 0. but we get an error for putting log 0
    if x <= 0. or x >= 1.:
        return 0
    else:
        return x*log(x)


# Mixed estimators
def micd(x, y, k=3, base=2, warning=True):
    """ If x is continuous and y is discrete, compute mutual information
    """

    overallentropy = entropy(x, k, base)
    n = len(y)
    word_dict = dict()
    for sample in y:
        word_dict[sample] = word_dict.get(sample, 0) + 1./n
    yvals = list(set(word_dict.keys()))

    mi = overallentropy
    for yval in yvals:
        xgiveny = [x[i] for i in range(n) if y[i] == yval]
        if k <= len(xgiveny) - 1:
            mi -= word_dict[yval]*entropy(xgiveny, k, base)
        else:
            if warning:
                print("Warning, after conditioning, on y={0} insufficient data. Assuming maximal entropy in this case.".format(yval))
            mi -= word_dict[yval]*overallentropy
    return mi  # units already applied


# Utility functions
def vectorize(scalarlist):
    """
    Turn a list of scalars into a list of one-d vectors
    """

    return [(x,) for x in scalarlist]


def shuffle_test(measure, x, y, z=False, ns=200, ci=0.95, **kwargs):
    """
    Shuffle test
    Repeatedly shuffle the x-values and then estimate measure(x,y,[z]).
    Returns the mean and conf. interval ('ci=0.95' default) over 'ns' runs, 'measure' could me mi,cmi,
    e.g. Keyword arguments can be passed. Mutual information and CMI should have a mean near zero.
    """

    xp = x[:]   # A copy that we can shuffle
    outputs = []
    for i in range(ns):
        random.shuffle(xp)
        if z:
            outputs.append(measure(xp, y, z, **kwargs))
        else:
            outputs.append(measure(xp, y, **kwargs))
    outputs.sort()
    return np.mean(outputs), (outputs[int((1.-ci)/2*ns)], outputs[int((1.+ci)/2*ns)])


# Internal functions
def avgdigamma(points, dvec):
    # This part finds number of neighbors in some radius in the marginal space
    # returns expectation value of <psi(nx)>
    N = len(points)
    tree = ss.cKDTree(points)
    avg = 0.
    for i in range(N):
        dist = dvec[i]
        # subtlety, we don't include the boundary point,
        # but we are implicitly adding 1 to kraskov def bc center point is included
        num_points = len(tree.query_ball_point(points[i], dist-1e-15, p=float('inf')))
        avg += digamma(num_points)/N
    return avg


def zip2(*args):
    # zip2(x,y) takes the lists of vectors and makes it a list of vectors in a joint space
    # E.g. zip2([[1],[2],[3]],[[4],[5],[6]]) = [[1,4],[2,5],[3,6]]
    return [sum(sublist, []) for sublist in zip(*args)]

In [7]:



def information_gain(f1, f2):
    """
    This function calculates the information gain, where ig(f1,f2) = H(f1) - H(f1|f2)

    Input
    -----
    f1: {numpy array}, shape (n_samples,)
    f2: {numpy array}, shape (n_samples,)

    Output
    ------
    ig: {float}
    """

    ig = entropyd(f1) - conditional_entropy(f1, f2)
    return ig


def conditional_entropy(f1, f2):
    """
    This function calculates the conditional entropy, where ce = H(f1) - I(f1;f2)

    Input
    -----
    f1: {numpy array}, shape (n_samples,)
    f2: {numpy array}, shape (n_samples,)

    Output
    ------
    ce: {float}
        ce is conditional entropy of f1 and f2
    """

    ce = entropyd(f1) - midd(f1, f2)
    return ce


def su_calculation(f1, f2):
    """
    This function calculates the symmetrical uncertainty, where su(f1,f2) = 2*IG(f1,f2)/(H(f1)+H(f2))

    Input
    -----
    f1: {numpy array}, shape (n_samples,)
    f2: {numpy array}, shape (n_samples,)

    Output
    ------
    su: {float}
        su is the symmetrical uncertainty of f1 and f2

    """

    # calculate information gain of f1 and f2, t1 = ig(f1,f2)
    t1 = information_gain(f1, f2)
    # calculate entropy of f1, t2 = H(f1)
    t2 = entropyd(f1)
    # calculate entropy of f2, t3 = H(f2)
    t3 = entropyd(f2)
    # su(f1,f2) = 2*t1/(t2+t3)
    su = 2.0*t1/(t2+t3)

    return su

In [27]:
df = pd.read_csv('/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/datasets/jshint.csv')
y = df.BUGS.values
X = df.drop(labels = ['BUGS'],axis = 1)
X = X.values

In [28]:
z = cfs(X,y)

[0] 0.0666871992367347
[1] 0.07547795182717584
[2] 0.07920970409078297
[3] 0.07920970409078297
[4] 0.07920970409078297
[5] 0.0793922940718235
[6] 0.07643423246731482
[7] 0.0795757277911569
[8] 0.0667821591264674
[9] 0.08013115027014027
[10] 0.07920970409078297
[11] 0.07172207954190864
[12] 0.07920970409078297
[13] 0.05954578076927013
[14] 0.06160311065985751
[15] 0.07583080762313633
[16] 0.07565396829090328
[17] 0.06754029590950446
[18] 0.07976001111062947
[19] 0.07920970409078297
[20] 0.07920970409078297
[21] 0.07920970409078297
[22] 0.0793922940718235
[23] 0.0749547826105712
[24] 0.0793922940718235
[25] 0.06747816446553463
[26] 0.0795757277911569
[27] 0.07920970409078297
[28] 0.07395865084559657
[29] 0.07920970409078297
[30] 0.0749547826105712
[31] 0.06401145703080774
[32] 0.07583080762313633
[33] 0.07530275250182912
[34] 0.041590663396005444
[35] 0.06660805222038053
[36] 0.07920970409078297
[37] 0.07583080762313633
[38] 0.07920970409078297
[39] 0.06401145703080774
[40] 0.07076281488

[9, 147, 6] 0.09064764431654317
[9, 147, 7] 0.09183756928576248
[9, 147, 8] 0.0902583413446039
[9, 147, 10] 0.09170740879553
[9, 147, 11] 0.08884165664298914
[9, 147, 12] 0.09170740879553
[9, 147, 13] 0.08612838345192093
[9, 147, 14] 0.08583355512486569
[9, 147, 15] 0.09037310673686293
[9, 147, 16] 0.09026744502831953
[9, 147, 17] 0.08741883996983185
[9, 147, 18] 0.09192115945287267
[9, 147, 19] 0.09170740879553
[9, 147, 20] 0.09170740879553
[9, 147, 21] 0.09170740879553
[9, 147, 22] 0.0917543468497871
[9, 147, 23] 0.08997416099278084
[9, 147, 24] 0.0917543468497871
[9, 147, 25] 0.08746424155267395
[9, 147, 26] 0.09180152611083546
[9, 147, 27] 0.09170740879553
[9, 147, 28] 0.08961211856290092
[9, 147, 29] 0.09170740879553
[9, 147, 30] 0.08997416099278084
[9, 147, 31] 0.08687437112784992
[9, 147, 32] 0.09037310673686293
[9, 147, 33] 0.09013268270137759
[9, 147, 34] 0.08107932359106748
[9, 147, 35] 0.08705647601260279
[9, 147, 36] 0.09170740879553
[9, 147, 37] 0.09037310673686293
[9, 147

[9, 147, 49, 53] 0.08859875580663107
[9, 147, 49, 54] 0.08624167340044384
[9, 147, 49, 55] 0.0889907546143117
[9, 147, 49, 56] 0.08872827340850119
[9, 147, 49, 57] 0.08531639369672928
[9, 147, 49, 58] 0.08531639369672928
[9, 147, 49, 59] 0.08531639369672928
[9, 147, 49, 60] 0.08531639369672928
[9, 147, 49, 61] 0.08531639369672928
[9, 147, 49, 62] 0.08531639369672928
[9, 147, 49, 63] 0.08531639369672928
[9, 147, 49, 64] 0.08531639369672928
[9, 147, 49, 65] 0.08531639369672928
[9, 147, 49, 66] 0.08531639369672928
[9, 147, 49, 67] 0.08531639369672928
[9, 147, 49, 68] 0.08531639369672928
[9, 147, 49, 69] 0.08531639369672928
[9, 147, 49, 70] 0.08531639369672928
[9, 147, 49, 71] 0.08531639369672928
[9, 147, 49, 72] 0.08531639369672928
[9, 147, 49, 73] 0.08531639369672928
[9, 147, 49, 74] 0.08531639369672928
[9, 147, 49, 75] 0.08531639369672928
[9, 147, 49, 76] 0.08531639369672928
[9, 147, 49, 77] 0.08531639369672928
[9, 147, 49, 78] 0.08531639369672928
[9, 147, 49, 79] 0.08531639369672928
[9

[9, 147, 49, 97, 27] 0.09105493292979402
[9, 147, 49, 97, 28] 0.08971094551519768
[9, 147, 49, 97, 29] 0.09105493292979402
[9, 147, 49, 97, 30] 0.08990901283288921
[9, 147, 49, 97, 31] 0.08809693807313325
[9, 147, 49, 97, 32] 0.0901874188440601
[9, 147, 49, 97, 33] 0.09001965935868982
[9, 147, 49, 97, 34] 0.08500934540981166
[9, 147, 49, 97, 35] 0.08800652344074147
[9, 147, 49, 97, 36] 0.09105493292979402
[9, 147, 49, 97, 37] 0.0901874188440601
[9, 147, 49, 97, 38] 0.09105493292979402
[9, 147, 49, 97, 39] 0.08809693807313325
[9, 147, 49, 97, 40] 0.08932580924013438
[9, 147, 49, 97, 41] 0.09119762196495781
[9, 147, 49, 97, 42] 0.09105493292979402
[9, 147, 49, 97, 43] 0.09105493292979402
[9, 147, 49, 97, 44] 0.09105493292979402
[9, 147, 49, 97, 45] 0.09108151558036166
[9, 147, 49, 97, 46] 0.08990901283288921
[9, 147, 49, 97, 47] 0.0911394443311244
[9, 147, 49, 97, 48] 0.08832209491023406
[9, 147, 49, 97, 50] 0.09105493292979402
[9, 147, 49, 97, 51] 0.0896281877047487
[9, 147, 49, 97, 52]

[9, 147, 49, 97, 144, 3] 0.09195957199666954
[9, 147, 49, 97, 144, 4] 0.09195957199666954
[9, 147, 49, 97, 144, 5] 0.09198200133456647
[9, 147, 49, 97, 144, 6] 0.09135528506457154
[9, 147, 49, 97, 144, 7] 0.09203002743851245
[9, 147, 49, 97, 144, 8] 0.0909466006509662
[9, 147, 49, 97, 144, 10] 0.09195957199666954
[9, 147, 49, 97, 144, 11] 0.09025167154757827
[9, 147, 49, 97, 144, 12] 0.09195957199666954
[9, 147, 49, 97, 144, 13] 0.08939335430934535
[9, 147, 49, 97, 144, 14] 0.0883389853150635
[9, 147, 49, 97, 144, 15] 0.09115373954781582
[9, 147, 49, 97, 144, 16] 0.09113394672202435
[9, 147, 49, 97, 144, 17] 0.08932133325519177
[9, 147, 49, 97, 144, 18] 0.09207825622988544
[9, 147, 49, 97, 144, 19] 0.09195957199666954
[9, 147, 49, 97, 144, 20] 0.09195957199666954
[9, 147, 49, 97, 144, 21] 0.09195957199666954
[9, 147, 49, 97, 144, 22] 0.09198200133456647
[9, 147, 49, 97, 144, 23] 0.09092372110423871
[9, 147, 49, 97, 144, 24] 0.09198200133456647
[9, 147, 49, 97, 144, 25] 0.08934421423754

[9, 147, 49, 97, 144, 205] 0.08884285099251235
[9, 147, 49, 97, 144, 206] 0.08844024848913702
[9, 147, 49, 97, 144, 207] 0.08844024848913702
[9, 147, 49, 97, 144, 208] 0.08844024848913702
[9, 147, 49, 97, 144, 209] 0.08844024848913702
[9, 147, 49, 97, 144, 210] 0.08844024848913702
[9, 147, 49, 97, 144, 211] 0.08844024848913702
[9, 147, 49, 97, 144, 212] 0.08844024848913702
[9, 147, 49, 97, 144, 213] 0.08844024848913702
[9, 147, 49, 97, 144, 214] 0.08844024848913702
[9, 147, 49, 97, 144, 215] 0.08844024848913702
[9, 147, 49, 97, 144, 216] 0.08844024848913702
[9, 147, 49, 97, 144, 217] 0.08844024848913702
[9, 147, 49, 97, 144, 218] 0.08419274734567098
[9, 147, 49, 97, 144, 219] 0.08844024848913702
[9, 147, 49, 97, 144, 220] 0.08844024848913702
[9, 147, 49, 97, 144, 221] 0.08844024848913702
[9, 147, 49, 97, 144, 222] 0.08844024848913702
[9, 147, 49, 97, 144, 223] 0.08844024848913702
[9, 147, 49, 97, 144, 224] 0.08844024848913702
[9, 147, 49, 97, 144, 225] 0.08844024848913702
[9, 147, 49, 

[9, 147, 49, 97, 144, 18, 132] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 133] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 134] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 135] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 136] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 137] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 138] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 139] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 140] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 141] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 142] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 143] 0.08949518694635823
[9, 147, 49, 97, 144, 18, 145] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 146] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 148] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 149] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 150] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 151] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 152] 0.08954746025594791
[9, 147, 49, 97, 144, 18, 153] 

[9, 147, 49, 97, 144, 18, 41, 59] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 60] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 61] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 62] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 63] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 64] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 65] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 66] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 67] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 68] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 69] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 70] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 71] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 72] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 73] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 74] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 75] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 76] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41

[9, 147, 49, 97, 144, 18, 41, 221] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 222] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 223] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 224] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 225] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 226] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 227] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 228] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 229] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 230] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 231] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 232] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 233] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 234] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 235] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 236] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 237] 0.08934608271008494
[9, 147, 49, 97, 144, 18, 41, 238] 0.08934608271008494
[9, 147, 4

[9, 147, 49, 97, 144, 18, 41, 8, 128] 0.08848107181567712
[9, 147, 49, 97, 144, 18, 41, 8, 129] 0.08889897915510614
[9, 147, 49, 97, 144, 18, 41, 8, 130] 0.08889897915510614
[9, 147, 49, 97, 144, 18, 41, 8, 131] 0.08889897915510614
[9, 147, 49, 97, 144, 18, 41, 8, 132] 0.08889897915510614
[9, 147, 49, 97, 144, 18, 41, 8, 133] 0.08889897915510614
[9, 147, 49, 97, 144, 18, 41, 8, 134] 0.08889897915510614
[9, 147, 49, 97, 144, 18, 41, 8, 135] 0.08889897915510614
[9, 147, 49, 97, 144, 18, 41, 8, 136] 0.08889897915510614
[9, 147, 49, 97, 144, 18, 41, 8, 137] 0.08889897915510614
[9, 147, 49, 97, 144, 18, 41, 8, 138] 0.08889897915510614
[9, 147, 49, 97, 144, 18, 41, 8, 139] 0.08889897915510614
[9, 147, 49, 97, 144, 18, 41, 8, 140] 0.08889897915510614
[9, 147, 49, 97, 144, 18, 41, 8, 141] 0.08889897915510614
[9, 147, 49, 97, 144, 18, 41, 8, 142] 0.08889897915510614
[9, 147, 49, 97, 144, 18, 41, 8, 143] 0.08893302412732683
[9, 147, 49, 97, 144, 18, 41, 8, 145] 0.08889897915510614
[9, 147, 49, 9